In [1]:
from __future__ import annotations

import argparse
import csv
import sys
import time
from datetime import datetime, timezone
from typing import Dict, Optional

try:
    import yfinance as yf
except Exception as e:
    print("Error: yfinance is required. Install with: pip install yfinance", file=sys.stderr)
    raise

# Friendly name -> default ticker helpers
NICKNAMES: Dict[str, str] = {
    "google": "GOOGL",     # Alphabet Class A
    "alphabet": "GOOGL",
    "googl": "GOOGL",
    "goog": "GOOG",        # Alphabet Class C
    "apple": "AAPL",
    "microsoft": "MSFT",
    "tesla": "TSLA",
    "amazon": "AMZN",
    "meta": "META",
    "facebook": "META",
    "nvidia": "NVDA",
}

def resolve_ticker(user_input: str) -> str:
    key = user_input.strip().lower()
    return NICKNAMES.get(key, user_input.strip().upper())

def utc_now_iso() -> str:
    return datetime.now(timezone.utc).isoformat()

def fetch_quote(ticker: str) -> Optional[dict]:
    """
    Fetch a consolidated quote dict using yfinance with sensible fallbacks.
    Returns None if nothing could be fetched.
    """
    t = yf.Ticker(ticker)

    # Try to use fast_info (fastest, minimal overhead)
    last = bid = ask = prev_close = volume = None
    currency = exchange = short_name = market_state = None

    try:
        fi = t.fast_info
        # Some fields may be missing per ticker
        last = fi.get("last_price", None)
        bid = fi.get("bid", None)
        ask = fi.get("ask", None)
        prev_close = fi.get("previous_close", None)
        volume = fi.get("last_volume", None) or fi.get("volume", None)
        currency = fi.get("currency", None)
        exchange = fi.get("exchange", None)
    except Exception:
        pass

    # Supplement with basic_info if available
    try:
        info = getattr(t, "info", {}) or {}
        currency = currency or info.get("currency")
        exchange = exchange or info.get("fullExchangeName") or info.get("exchange")
        short_name = info.get("shortName") or info.get("longName")
        market_state = info.get("marketState")
        if prev_close is None:
            prev_close = info.get("regularMarketPreviousClose")
        if last is None:
            last = info.get("regularMarketPrice")
        if bid is None:
            bid = info.get("bid")
        if ask is None:
            ask = info.get("ask")
        if volume is None:
            volume = info.get("regularMarketVolume")
    except Exception:
        pass

    # If still missing 'last', try minute history as a final fallback
    if last is None:
        try:
            df = t.history(period="1d", interval="1m", prepost=True)
            if df is not None and not df.empty:
                last = float(df["Close"].iloc[-1])
        except Exception:
            pass

    # If still nothing, fail
    if last is None and bid is None and ask is None and prev_close is None:
        return None

    # Compute change/percent if possible
    change = None
    change_pct = None
    try:
        if last is not None and prev_close not in (None, 0):
            change = float(last) - float(prev_close)
            change_pct = (change / float(prev_close)) * 100.0
    except Exception:
        pass

    return {
        "last": _to_float(last),
        "bid": _to_float(bid),
        "ask": _to_float(ask),
        "prev_close": _to_float(prev_close),
        "change": _to_float(change),
        "change_pct": _to_float(change_pct),
        "volume": _to_int(volume),
        "currency": currency,
        "exchange": exchange,
        "short_name": short_name,
        "market_state": market_state,
    }

def _to_float(x):
    try:
        return None if x is None else float(x)
    except Exception:
        return None

def _to_int(x):
    try:
        return None if x is None else int(x)
    except Exception:
        return None

def run_forever(ticker: str, interval: int, out_csv: str, quiet: bool):
    # Prepare CSV and write header if needed
    write_header = False
    try:
        with open(out_csv, "r", encoding="utf-8") as _:
            pass
    except FileNotFoundError:
        write_header = True

    with open(out_csv, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if write_header:
            writer.writerow([
                "timestamp_utc",
                "ticker",
                "last",
                "bid",
                "ask",
                "change",
                "change_pct",
                "prev_close",
                "volume",
                "market_state",
                "currency",
                "exchange",
                "short_name",
            ])

        while True:
            ts = utc_now_iso()
            q = None
            err = None
            try:
                q = fetch_quote(ticker)
            except Exception as e:
                err = e

            if q:
                writer.writerow([
                    ts,
                    ticker,
                    _fmt(q["last"]),
                    _fmt(q["bid"]),
                    _fmt(q["ask"]),
                    _fmt(q["change"]),
                    _fmt(q["change_pct"]),
                    _fmt(q["prev_close"]),
                    q.get("volume") if q.get("volume") is not None else "",
                    q.get("market_state") or "",
                    q.get("currency") or "",
                    q.get("exchange") or "",
                    q.get("short_name") or "",
                ])
                f.flush()
                if not quiet:
                    _print_console(ts, ticker, q)
            else:
                msg = f"{ts}  {ticker}: quote unavailable"
                if err:
                    msg += f" ({err})"
                print(msg, file=sys.stderr)

            time.sleep(max(1, interval))


def _fmt(x):
    if x is None:
        return ""
    try:
        return f"{float(x):.6f}"
    except Exception:
        return str(x)

def _print_console(ts, ticker, q):
    last = q.get("last")
    chg = q.get("change")
    chgp = q.get("change_pct")
    bid = q.get("bid")
    ask = q.get("ask")
    mkt = q.get("market_state") or ""
    ccy = q.get("currency") or ""
    parts = [
        ts,
        f"{ticker}",
        f"last={_fmt(last)}",
        f"bid/ask={_fmt(bid)}/{_fmt(ask)}",
        f"Δ={_fmt(chg)} ({_fmt(chgp)}%)",
        f"{mkt} {ccy}".strip(),
    ]
    print("  ".join(p for p in parts if p))

def parse_args(argv=None):
    p = argparse.ArgumentParser(description="24/7 quote logger using yfinance")
    p.add_argument("--ticker", default="GOOGL", help="Ticker symbol or nickname (default: GOOGL). Examples: MSFT, AAPL, TSLA, google")
    p.add_argument("--interval", type=int, default=30, help="Polling interval in seconds (default: 30)")
    p.add_argument("--out", default="quotes.csv", help="CSV output path (default: quotes.csv)")
    p.add_argument("--quiet", action="store_true", help="Suppress console prints (still writes CSV)")
    return p.parse_args(argv)

def main(argv=None):
    args = parse_args(argv)
    ticker = resolve_ticker(args.ticker)
    print(f"Starting watcher for {ticker} every {args.interval}s. Writing to {args.out}. Press Ctrl+C to stop.", file=sys.stderr)
    run_forever(ticker, args.interval, args.out, args.quiet)



In [2]:
aapl = yf.Ticker("aapl")
news = aapl.news

news

[{'id': 'a6c02fd5-7b13-33fa-878c-4ba88f6f2f3d',
  'content': {'id': 'a6c02fd5-7b13-33fa-878c-4ba88f6f2f3d',
   'contentType': 'VIDEO',
   'title': 'Nvidia, Tesla dominate Dec. trading as 2025 options demand surges',
   'description': '<p>Cboe vice president of market intelligence Henry Schwartz joins Market Domination with Yahoo Finance Data and Markets Editor <a data-i13n="cpos:1;pos:1" href="https://finance.yahoo.com/author/jared-blikre/">Jared Blikre</a> to discuss the state of the options landscape.</p>\n<p>Watch the video above to hear Schwartz\'s outlook for the options market in 2026.</p>\n<p>To watch more expert insights and analysis on the latest market action, check out more <a data-i13n="cpos:2;pos:1" href="https://finance.yahoo.com/videos/series/market-domination/">Market Domination</a>.</p>',
   'summary': "Cboe vice president of market intelligence Henry Schwartz joins Market Domination with Yahoo Finance Data and Markets Editor Jared Blikre to discuss the state of the op

In [3]:
aapl = yf.Ticker("GOOG")
news = aapl.news

news

[{'id': '18ff5232-3703-44b7-a548-65c68fbea3cc',
  'content': {'id': '18ff5232-3703-44b7-a548-65c68fbea3cc',
   'contentType': 'STORY',
   'title': 'What’s next for EVs after a year of unraveling',
   'description': '',
   'summary': 'In 2026, a hybrid approach is the recalibration the auto industry is banking on.',
   'pubDate': '2025-12-20T11:00:45Z',
   'displayTime': '2025-12-20T12:36:27Z',
   'isHosted': True,
   'bypassModal': False,
   'previewUrl': None,
   'thumbnail': {'originalUrl': 'https://s.yimg.com/os/creatr-uploaded-images/2025-12/57fd4a70-dcee-11f0-9f57-29827ad768a5',
    'originalWidth': 8034,
    'originalHeight': 5347,
    'caption': '',
    'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/UxJm1q09HbjiyLKI2uqyYw--~B/aD01MzQ3O3c9ODAzNDthcHBpZD15dGFjaHlvbg--/https://s.yimg.com/os/creatr-uploaded-images/2025-12/57fd4a70-dcee-11f0-9f57-29827ad768a5',
      'width': 8034,
      'height': 5347,
      'tag': 'original'},
     {'url': 'https://s.yimg.com/uu/api/res

In [4]:
aapl = yf.Ticker("qqq")
news = aapl.news

news

[{'id': '6d5638d5-d52d-38b2-a278-5694b038fc4e',
  'content': {'id': '6d5638d5-d52d-38b2-a278-5694b038fc4e',
   'contentType': 'STORY',
   'title': 'Is IVV or QQQ a Better Choice for Investors? How These Popular ETFs Compare on Risk and Returns',
   'description': '',
   'summary': "Expense ratio, sector mix, and yield set these two ETFs apart. See how their differences may shape your portfolio's risk and income profile.",
   'pubDate': '2025-12-20T12:20:06Z',
   'displayTime': '2025-12-20T12:20:06Z',
   'isHosted': True,
   'bypassModal': False,
   'previewUrl': None,
   'thumbnail': {'originalUrl': 'https://media.zenfs.com/en/motleyfool.com/6b28a065be0b5031477267afbc8e62a9',
    'originalWidth': 1401,
    'originalHeight': 1251,
    'caption': '',
    'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/Np9o3bh8VC1sVtimeVtyug--~B/aD0xMjUxO3c9MTQwMTthcHBpZD15dGFjaHlvbg--/https://media.zenfs.com/en/motleyfool.com/6b28a065be0b5031477267afbc8e62a9',
      'width': 1401,
      'heigh

In [5]:
aapl= yf.Ticker("aapl")
aapl_stock_info = aapl.info

aapl_stock_info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '(408) 996-1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple Vision Pro, Apple TV, Apple Watch, Beats products, and HomePod, as well as Apple branded and third-party accessories. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download app

In [6]:
goog = yf.Ticker("GOOG")
goog_stock_info = goog.info

set(aapl_stock_info.keys()).difference(set(goog_stock_info.keys()))

{'auditRisk',
 'boardRisk',
 'compensationRisk',
 'fiveYearAvgDividendYield',
 'governanceEpochDate',
 'irWebsite',
 'overallRisk',
 'shareHolderRightsRisk',
 'shortPercentOfFloat'}

In [7]:
aapl_historical = aapl.history(start="2020-06-02", end="2020-06-07", interval="1m")
aapl_historical

$AAPL: possibly delisted; no price data found  (1m 2020-06-02 -> 2020-06-07) (Yahoo error = "1m data not available for startTime=1591070400 and endTime=1591502400. The requested range must be within the last 30 days.")


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [10]:
aapl.earnings_estimate

,avg,low,high,yearAgoEps,numberOfAnalysts,growth
period,,,,,,
0q,1.75840,1.63,1.82,0.97000,26,0.8128
+1q,2.46401,2.29,2.67,2.40000,22,0.0267
0y,7.38321,7.25,7.47,6.08000,35,0.2143
+1y,7.94575,7.13,9.00,7.38321,38,0.0762


In [13]:
result = aapl.get_earnings_estimate()
print(result)

            avg   low  high  yearAgoEps  numberOfAnalysts  growth
period                                                           
0q      1.75840  1.63  1.82     0.97000                26  0.8128
+1q     2.46401  2.29  2.67     2.40000                22  0.0267
0y      7.38321  7.25  7.47     6.08000                35  0.2143
+1y     7.94575  7.13  9.00     7.38321                38  0.0762
